## Explore Data

This notebook uses the ```napari-easy-augment-batch-dl``` widget to explore and label the data.  If we have a model trained we can also predict using that model. 

In [1]:
import os
import numpy as np
import napari
from napari_easy_augment_batch_dl import easy_augment_batch_dl

raster_geometry not imported.  This is only needed for the ellipsoid rendering in apply_stardist


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.18 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [ ]:

viewer = napari.Viewer()

batch_dl = easy_augment_batch_dl.NapariEasyAugmentBatchDL(viewer, label_only = False)

viewer.window.add_dock_widget(
    batch_dl
)

data_path = r'C:\Users\bnort\work\ImageJ2022\tnia\notebooks-and-napari-widgets-for-dl\data'
data_path = r'/home/bnorthan/code/i2k/tnia/notebooks-and-napari-widgets-for-dl/data'
parent_path = os.path.join(data_path, 'ladybugs1')
model_path = os.path.join(parent_path, 'models')

model_name = 'ladybug_first_try'
model_type = "Stardist Model"

model_name = 'cellpose_for_ladybugs'
model_type = "CellPose Instance Model"

batch_dl.load_image_directory(parent_path)


if model_name is not None:
    batch_dl.network_architecture_drop_down.setCurrentText(model_type)
    batch_dl.deep_learning_project.set_pretrained_model(os.path.join(model_path, model_name), model_type)

    '''
    model = batch_dl.deep_learning_project.models[model_type]
    model.prob_thresh = -1
    model.flow_thresh = 0.4
    model.chan_segment = 2
    model.chan2 = 3
    '''    

    widget = batch_dl.param_widgets[model_type]
    widget.sync_with_model()





INFO:cellpose.io:WRITING LOG OUTPUT TO /home/bnorthan/.cellpose/run.log
INFO:cellpose.io:
cellpose version: 	3.0.10 
platform:       	linux 
python version: 	3.12.4 
torch version:  	2.4.0+cu121


found class  CellPoseInstanceModel


INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU
INFO:cellpose.models:>>>> loading model /home/bnorthan/code/i2k/tnia/notebooks-and-napari-widgets-for-dl/data/ladybugs1/models/cellpose_for_ladybugs
/home/bnorthan/mambaforge/envs/easy_augment_pytorch/lib/python3.12/site-packages/cellpose/resnet_torch.py:276: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.seriali

INFO:cellpose.models:>> cyto3 << model set to be used
INFO:cellpose.models:Downloading: "https://www.cellpose.org/models/cyto3" to /home/bnorthan/.cellpose/models/cyto3

100%|██████████| 25.3M/25.3M [00:02<00:00, 9.74MB/s]
INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU
INFO:cellpose.models:>>>> loading model /home/bnorthan/.cellpose/models/cyto3
/home/bnorthan/mambaforge/envs/easy_augment_pytorch/lib/python3.12/site-packages/cellpose/resnet_torch.py:276: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during 